In [2]:
import os 
import math
import pandas as pd
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, auc, roc_curve
# import seaborn as sns
%matplotlib inline

data_folder = '../data/'

# Data Descroption
Plot the data points location distribution

In [6]:
columns = ['RELI', 'EAST', 'NORTH']

location_data = pd.read_csv(os.path.join(data_folder, 'data_with_neighbour.csv'), usecols=columns)
location_data.set_index('RELI', inplace=True)
location_data.head()

,EAST,NORTH
RELI,,
73012621,2730100,1262100
71932615,2719300,1261500
74942622,2749400,1262200
73102619,2731000,1261900
73342625,2733400,1262500


# Logistic Regression

### A) Load the dataset and convert categorical features to a suitable numerical representation (use dummy-variable encoding). 
- Split the data into a training set (80%) and a test set (20%). Pair each feature vector with the corresponding label, i.e., whether the outcome_type is adoption or not. 
- Standardize the values of each feature in the data to have mean 0 and variance 1.

In [9]:
columns = ['LU4', 'LC4', 'LU3', 'LC3', 'LU2', 'LC2', 'LU1', 'LC1', 'nbr1_LU3', 'nbr1_LC3', 'nbr1_LU2', 'nbr1_LC2', 'nbr1_LU1',
           'nbr1_LC1', 'nbr2_LU3', 'nbr2_LC3', 'nbr2_LU2', 'nbr2_LC2', 'nbr2_LU1', 'nbr2_LC1', 'nbr3_LU3', 'nbr3_LC3',
           'nbr3_LU2', 'nbr3_LC2', 'nbr3_LU1', 'nbr3_LC1', 'nbr4_LU3', 'nbr4_LC3', 'nbr4_LU2', 'nbr4_LC2', 'nbr4_LU1',
           'nbr4_LC1', 'nbr5_LU3', 'nbr5_LC3', 'nbr5_LU2', 'nbr5_LC2', 'nbr5_LU1', 'nbr5_LC1', 'nbr6_LU3', 'nbr6_LC3',
           'nbr6_LU2', 'nbr6_LC2', 'nbr6_LU1', 'nbr6_LC1', 'nbr7_LU3', 'nbr7_LC3', 'nbr7_LU2', 'nbr7_LC2', 'nbr7_LU1',
           'nbr7_LC1', 'nbr8_LU3', 'nbr8_LC3', 'nbr8_LU2', 'nbr8_LC2', 'nbr8_LU1', 'nbr8_LC1']

original_data = pd.read_csv(os.path.join(data_folder, 'data_with_neighbour.csv'), index_col=0)
original_data = original_data[columns]
original_data.head()

,LU4,LC4,LU3,LC3,LU2,LC2,LU1,LC1,nbr1_LU3,nbr1_LC3,...,nbr7_LU2,nbr7_LC2,nbr7_LU1,nbr7_LC1,nbr8_LU3,nbr8_LC3,nbr8_LU2,nbr8_LC2,nbr8_LU1,nbr8_LC1
73012621,LU107,LU11,LU107,LU11,LU107,LU11,LU107,LU11,LU107,LU12,...,LU223,LU21,LU222,LU21,LU201,LU47,LU201,LU47,LU201,LU47
71932615,LU124,LU21,LU124,LU21,LU124,LU21,LU124,LU21,LU223,LU21,...,LU222,LU21,LU222,LU21,LU222,LU21,LU222,LU21,LU222,LU21
74942622,LU122,LU11,LU122,LU11,LU122,LU11,LU122,LU11,LU223,LU21,...,LU122,LU31,LU222,LU21,LU222,LU21,LU222,LU21,LU222,LU21
73102619,LU201,LU21,LU201,LU21,LU201,LU44,LU201,LU44,LU221,LU21,...,LU107,LU11,LU107,LU11,LU221,LU21,LU221,LU21,LU221,LU21
73342625,LU222,LU21,LU223,LU21,LU223,LU21,LU223,LU21,LU301,LU42,...,LU201,LU44,LU201,LU44,LU222,LU21,LU222,LU21,LU222,LU21


# Random Forest 